In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install ultralytics
!pip install collections

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from transformers import DetrImageProcessor, DetrForObjectDetection
import torch
from PIL import Image
import requests
import os
from tqdm import tqdm
def object_detection(img_folder):
  final_output = []
  total_img = len(os.listdir(img_folder))
  for i in tqdm(range(total_img)):
    img_path = img_folder + str(i) + '.png'
    image = Image.open(img_path)
    # you can specify the revision tag if you don't want the timm dependency
    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50", revision="no_timm")
    model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50", revision="no_timm")

    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)

    # convert outputs (bounding boxes and class logits) to COCO API
    # let's only keep detections with score > 0.9
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.7)[0]
    dict_ = {}
    detect_cls = []
    detect_box = []
    for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
      box = [round(i, 2) for i in box.tolist()]

      detect_cls.append(model.config.id2label[label.item()])
      detect_box.append(box)
    dict_['detect cls'] = detect_cls
    dict_['detect box'] = detect_box
    final_output.append(dict_)
  return final_output

In [ ]:
import json
for i in range(1,5):
  v15_file_path = '/content/drive/MyDrive/New_exp/sd1_5/exp{}/'.format(i)
  detect_result = object_detection(v15_file_path)
  with open('/content/drive/MyDrive/Error_Detection/v1-5/exp{}.json'.format(i), 'w') as f:
    json.dump(detect_result, f, indent=4)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/6.60k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/167M [00:00<?, ?B/s]

In [ ]:
import json
for i in range(2,3):
  for j in range(1,5):
    v15_file_path = '/content/drive/MyDrive/New_exp/sd1_5/ab{}/exp{}/'.format(i, j)
    detect_result = object_detection(v15_file_path)
    save_path = '/content/drive/MyDrive/Error_Detection/v1-5/ab{}/exp{}.json'.format(i, j)
    with open('/content/drive/MyDrive/Error_Detection/v1-5/ab{}/exp{}.json'.format(i, j), 'w') as f:
      json.dump(detect_result, f, indent=4)

100%|██████████| 543/543 [20:25<00:00,  2.26s/it]


In [ ]:
path1 = '/content/drive/MyDrive/New_exp/sd1_5/quick_test_ab1/'
path2 = '/content/drive/MyDrive/New_exp/sd1_5/quick_test_ab2/'

In [ ]:
path0 = '/content/drive/MyDrive/New_exp/sd1_5/quick_test/'

In [ ]:
import json
detect_result0 = object_detection(path0)
with open('/content/drive/MyDrive/Error_Detection/v1-5/quick_test.json', 'w') as f:
  json.dump(detect_result0, f, indent=4)
# detect_result1 = object_detection(path1)
# detect_result2 = object_detection(path2)
# with open('/content/drive/MyDrive/Error_Detection/v1-5/quick_test_ab1.json', 'w') as f:
#   json.dump(detect_result1, f, indent=4)
# with open('/content/drive/MyDrive/Error_Detection/v1-5/quick_test_ab2.json', 'w') as f:
#   json.dump(detect_result2, f, indent=4)


100%|██████████| 100/100 [02:46<00:00,  1.66s/it]
